In [1]:
import os

In [2]:
from dotenv import load_dotenv
load_dotenv()

'7f8a6915'

In [4]:
from langchain_core.messages import (
    BaseMessage,
    HumanMessage,
    ToolMessage,
)
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from langgraph.graph import END, StateGraph, START

In [5]:
def create_agent(llm, tools, system_message: str):
    """Create an Agent"""
    prompt = ChatPromptTemplate.from_messages([
        (
            "system",
            """You are a helpful AI assistant, collabrating with other assistants.
            Use the provided tools to progress towards answering the question.
            If you are unable to fully answer, that's OK, another assistant with different tools will help where you left off.
            Execute what you can to make progress.
            If you or any of the other assistants have the final answer or deliverable,
            prefix your response with FINAL ANSWER so the team knows to stop.
            You have access to the following tools: {tool_names}.\n{system_message},
            
            """
        ),
        MessagesPlaceholder(variable_name="messages")
    ])
    prompt=prompt.partial(system_message=system_message)
    prompt=prompt.partial(tool_names=", ".join([tool.name for tool in tools]))
    return prompt | llm.bind_tools(tools)


# Define Tools

In [9]:
from typing import Annotated
from langchain_community.tools.tavily_search import TavilySearchResults

tavily_tool = TavilySearchResults(max_resutls=5)

In [11]:
from langchain_core.tools import tool
from langchain_experimental.utilities import PythonREPL

repl = PythonREPL()

@tool
def python_repl(code: Annotated[str, "The python code to execute to generate your chart."]):
    try:
        result = repl.run(code)
    except BaseException as e:
        return f"Failed to execute. Error: {repr(e)}"
    result_str = f"Successfully executed:\n```python\n{code}\n````\nStdout: {result}"
    return (
        result_str + "\n\nIf you have completed all tasks, respond with FINAL ANSWER."
    )

ValueError: Function must have a docstring if description not provided.